In [19]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

dataset_dir = "melanoma_cancer_dataset"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")


In [ ]:

img_size = 224
batch_size = 32
validation_split = 0.1
epochs_finetune = 15
epochs_initial = 15
seed = 42

dataset_dir = "melanoma_cancer_dataset"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")


train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode="binary"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode="binary"
)


AUTOTUNE = tf.data.AUTOTUNE

train_ds = (
    train_ds
    .map(lambda x, y: (preprocess_input(x), y))
    .cache()
    .shuffle(1000)
    .prefetch(buffer_size=AUTOTUNE)
)

val_ds = (
    val_ds
    .map(lambda x, y: (preprocess_input(x), y))
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)


labels = [int(y.numpy()) for _, y in train_ds.unbatch()]
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(labels),
    y=labels
)
class_weight_dict = dict(enumerate(class_weights))
print("Class Weights:", class_weight_dict)


test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode="binary",
    shuffle=False
)

test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y)).prefetch(buffer_size=AUTOTUNE)

Found 9605 files belonging to 2 classes.
Using 8645 files for training.
Found 9605 files belonging to 2 classes.
Using 960 files for validation.


/var/folders/l5/7wz4c_fn2wg4t__87s7y1_rr0000gn/T/ipykernel_20679/2455520452.py:53: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels = [int(y.numpy()) for _, y in train_ds.unbatch()]


Class Weights: {0: np.float64(0.9565169285240097), 1: np.float64(1.0476248182258847)}
Found 1000 files belonging to 2 classes.


2025-05-27 00:22:25.144703: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [21]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(inputs=base_model.input, outputs=output)


In [ ]:
print("Training with frozen EfficientNetB0")
base_model.trainable = False
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    os.path.join(dataset_dir, "model_frozen_best.keras"),
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


model.fit(
    train_ds,
    epochs=epochs_initial,
    validation_data=val_ds,
    callbacks=[early_stop, checkpoint],
    class_weight=class_weight_dict
)

model.save(os.path.join(dataset_dir, "model_frozen.keras"))


Training with frozen EfficientNetB0...
Epoch 1/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 709ms/step - accuracy: 0.6489 - loss: 0.6335
Epoch 1: val_accuracy improved from -inf to 0.86979, saving model to melanoma_cancer_dataset/model_frozen_best.keras
271/271 ━━━━━━━━━━━━━━━━━━━━ 220s 792ms/step - accuracy: 0.6492 - loss: 0.6333 - val_accuracy: 0.8698 - val_loss: 0.4146
Epoch 2/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 680ms/step - accuracy: 0.8193 - loss: 0.4419
Epoch 2: val_accuracy improved from 0.86979 to 0.88333, saving model to melanoma_cancer_dataset/model_frozen_best.keras
271/271 ━━━━━━━━━━━━━━━━━━━━ 207s 766ms/step - accuracy: 0.8193 - loss: 0.4418 - val_accuracy: 0.8833 - val_loss: 0.3388
Epoch 3/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 748ms/step - accuracy: 0.8529 - loss: 0.3701
Epoch 3: val_accuracy improved from 0.88333 to 0.88542, saving model to melanoma_cancer_dataset/model_frozen_best.keras
271/271 ━━━━━━━━━━━━━━━━━━━━ 224s 824ms/step - accuracy: 0.8529 - loss: 0.3701 - val_accuracy: 0.8

In [ ]:
print("Fine-tuning EfficientNetB0")
base_model.trainable = True 

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

checkpoint_path = os.path.join(dataset_dir, "model_finetuned_best.keras")

model.fit(
    train_ds,
    epochs=epochs_finetune,
    validation_data=val_ds,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True, verbose=1),
        ModelCheckpoint(checkpoint_path, monitor="val_accuracy", save_best_only=True, verbose=1)
    ],
    class_weight=class_weight_dict
)

model.save(os.path.join(dataset_dir, "model_finetuned.keras"))


Fine-tuning EfficientNetB0...
Epoch 1/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7548 - loss: 0.4872
Epoch 1: val_accuracy improved from -inf to 0.91146, saving model to melanoma_cancer_dataset/model_finetuned_best.keras
271/271 ━━━━━━━━━━━━━━━━━━━━ 975s 4s/step - accuracy: 0.7550 - loss: 0.4869 - val_accuracy: 0.9115 - val_loss: 0.2455
Epoch 2/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8878 - loss: 0.2896
Epoch 2: val_accuracy improved from 0.91146 to 0.92604, saving model to melanoma_cancer_dataset/model_finetuned_best.keras
271/271 ━━━━━━━━━━━━━━━━━━━━ 947s 3s/step - accuracy: 0.8878 - loss: 0.2896 - val_accuracy: 0.9260 - val_loss: 0.2036
Epoch 3/15
271/271 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9021 - loss: 0.2439
Epoch 3: val_accuracy improved from 0.92604 to 0.93333, saving model to melanoma_cancer_dataset/model_finetuned_best.keras
271/271 ━━━━━━━━━━━━━━━━━━━━ 910s 3s/step - accuracy: 0.9021 - loss: 0.2438 - val_accuracy: 0.9333 - val_loss: 0.

In [2]:
from tensorflow.keras.models import load_model

model_path = "melanoma_cancer_dataset/model_finetuned_best.keras"

model = load_model(model_path)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications.efficientnet import preprocess_input

img_size = 224
batch_size = 32
test_dir = "melanoma_cancer_dataset/test"

AUTOTUNE = tf.data.AUTOTUNE

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(img_size, img_size),
    batch_size=batch_size,
    label_mode='binary',
    shuffle=False
)

test_ds = test_ds.map(lambda x, y: (preprocess_input(x), y)).prefetch(AUTOTUNE)


Found 1000 files belonging to 2 classes.


In [4]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

threshold = 0.5

y_true = np.concatenate([y.numpy() for _, y in test_ds], axis=0)

probs = model.predict(test_ds)
preds = (probs > threshold).astype(int).flatten()

# Results
print("Confusion Matrix:")
print(confusion_matrix(y_true, preds))

print("\nClassification Report:")
print(classification_report(y_true, preds, target_names=["Benign", "Malignant"]))


2025-05-27 09:03:03.911411: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


32/32 ━━━━━━━━━━━━━━━━━━━━ 24s 723ms/step
Confusion Matrix:
[[477  23]
 [ 58 442]]

Classification Report:
              precision    recall  f1-score   support

      Benign       0.89      0.95      0.92       500
   Malignant       0.95      0.88      0.92       500

    accuracy                           0.92      1000
   macro avg       0.92      0.92      0.92      1000
weighted avg       0.92      0.92      0.92      1000



In [12]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(y_true, probs)

print(f"\nAUC-ROC Score: {auc_score:.4f}")



AUC-ROC Score: 0.9789
